### 신경망 정의하기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 입력 채널 1개(흑백) 
        # 출력 채널 6개
        # 5x5 크기의 커널을 사용하는 첫번째 합성곱 층
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 입력 채널 6개
        # 출력 채널 16개
        # 5x5 크기의 커널을 사용하는 두번째 합성곱 층
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 입력 틍징의 수가 16 * 5 * 5인 FC층 (120개의 출력 뉴런)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # 입력 뉴련이 120개, 출력 뉴런이 84개인 FC층
        self.fc2 = nn.Linear(120, 84) 
        # 입력 뉴련이 84개, 출력 뉴런이 10개인 FC층
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, input):
        # 입력 이미지에 5x5 커널로 합성곱 연산 수행
        c1 = F.relu(self.conv1(input))
        s2 = F.max_pool2d(c1, (2, 2))
        c3 = F.relu(self.conv2(s2))
        s4 = F.max_pool2d(c3, 2)
        s4 = torch.flatten(s4)
        f5 = F.relu(self.fc1(s4))
        f6 = F.relu(self.fc2(f5))
        
        output = self.fc3(f6)
        return output
    

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수만 정의하고 나면, 변화도를 계산하는 backward 함수는 autograd를 사용하여 자동으로 정의된다.

---

학습 가능한 파라미터들은 net.parameters()를 통해 반환 가능하다.

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


해당 net의 input 예상 사이즈는 32x32이다. (MNIST의 이미지 사이즈는 32x32)

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([-0.0817,  0.0461,  0.1515,  0.0404,  0.1466,  0.0865,  0.1450,  0.0975,
        -0.0162, -0.0794], grad_fn=<ViewBackward0>)


모든 매개변수의 gradient buffer를 0으로 설정하고, 무작위 값으로 역전파를 진행한다.

gradient buffer

- 신경망의 각 가중치에 대해 계산된 기울기를 임시로 저장하는 공간
- 역전파가 끝난 후, 이 버퍼의 값은 최적화 알고리즘(SGD, Adam etc.)에 의해 사용되어 가중치를 업데이트
- 매 파라미터마다 기울기 버퍼가 있어 한 번의 역전파에 대해 계산된 모든 기울기가 이 버퍼에 저장됨

여기서 '무작위 값(임의의 기울기)으로 역전파를 수행하는 이유

- 일반적인 학습에서는 실제 손실 함수를 기준으로 계산된 기울기를 사용 but 이 경우에는 실험/학습 전 테스트용이기 때문에 의도적으로 무작위 기울기를 사용

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([1, 10]) and output[0] has a shape of torch.Size([10]).

### 손실 함수

손실 함수는 (output, target)을 쌍으로 받아, target으로부터 output이 얼마나 멀리있는지를 계산한다.

In [6]:
# MSELoss 예시

output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.4241, grad_fn=<MseLossBackward0>)


/Users/gabeen/anaconda3/envs/AI_project/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 10])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [7]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

### 역전파

In [8]:
net.zero_grad() # 모든 파라미터의 gradient buffer를 zero로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0310,  0.0044, -0.0166,  0.0030,  0.0187,  0.0080])


### 가중치 갱신

가장 단순한 갱신 규칙(update rule)은 SGD(Stochastic Gradient Descent)이다.

In [ ]:
weight = weight - learning_rate * gradient

위 식을 코드로 구현한 것

In [10]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

There was a problem when trying to write in your cache folder (/home/huggingface_cache/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.
/Users/gabeen/anaconda3/envs/AI_project/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 10])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
